## Data process
This script is used for train/test/valid dataset generation for GNHS related data.  
Phase 1 patients data is split into train/test(9/1) where phase 4 patients data is used for validation.

In [1]:
import os
import sys

os.chdir("..")
sys.path.append(r"D:\GitHub\GNHS\src")

In [2]:
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import VarianceThreshold
from utils.utils import sub_label_generator
from models.feature_selection import distribution_deviation
import snf
pd.set_option("display.max_columns", None)

seed = 0
np.random.seed(seed)

In [3]:
root_dir = Path(r"D:\GitHub\GNHS").resolve()

# protein matrix data baseline
prot_data_baseline = root_dir / "data/raw_data/20220304_GNHS1-3_uniquesample_inculde_inf.csv" 
prot_data_perspective = root_dir/ "data/raw_data/20220220_GNHS4_matrix_inf.csv"

# clinic data
clinic_data = root_dir/ "data/raw_data/20220106_GNHS1-4_patient_inf_include_TC.csv"

# output directory
TRANSFORM = root_dir/ "data/transformed_data/2022"

### Load protein matrix data  
- phase 1 for model development, phase 4 for model validation
- excluded columns: "time1", "pat", "metsscore", "menopau_age"
- met (metabolic syndrome)labels: "waist", "fbg", "tg", "hdl", "sbpdbp","mets"
- dm (diabetes mellitus) label: "dm_cl"
- hyper (hypertension) label: "hyper_cl"
- dyslipi (dyslipidemia) labels: "tg_cl", "tc_cl", "hdl_cl", "ldl_cl", "dys_cl"
- included clinic measures: "age", "sex", "BMI", "wc", "Glu", "TG", "TC", "HDL", "LDL", "SBP", "DBP"
- records with values more than 999 represent value missing (will be replaced with np.NaN)
- med fields will be imputed with 0 if related clinic diagnosis is 0
- med fields with values of 5 will be replace with 0 and fields with values of 2-4 will be replaced with 1, the rest remains NA

In [ ]:
exclude_columns = ["time1", "pat", 
                   "metsscore", "metsscoref2", "metsscoref3",
                   "menopau_age", "menopau_agef2", "menopau_agef3",
                   "agef2", "agef3",
                   "BMIf2", "BMIf3",
                   "wcf2", "wcf3",
                   "Gluf2",  "Gluf3",
                   "TGf2","TGf3",
                   "TCf2", "TCf3",
                   "HDLf2", "HDLf3",
                   "LDLf2", "LDLf3",
                   "DBPf2", "DBPf3",
                   "SBPf2", "SBPf3",
                   "TCf1"] # cohort 2 & cohort 3 clinic data will not be included in model development in the context of 10 years ahead prediction
clinic_measures = ["age", "sex", "BMI", "wc", "Glu", "TG", "TC", "HDL", "LDL", "DBP", "SBP"] # baseline only
med_conditions = ["DM_med", "DM_medf2", "DM_medf3",
                  "dyslipi_med", "dyslipi_medf2", "dyslipi_medf3",
                  "hyper_med", "hyper_medf2", "hyper_medf3"]
mets_label = ["waist", "waistf2", "waistf3",
              "fbg", "fbgf2", "fbgf3",
              "tg", "tgf2", "tgf3",
              "hdl", "hdlf2", "hdlf3",
              "sbpdbp", "sbpdbpf2", "sbpdbpf3",
              "mets", "metsf2", "metsf3"]
dm_label = ["dm_cl", "dm_clf2", "dm_clf3"]
hyper_label = ["hyper_cl", "hyper_clf2", "hyper_clf3"]
dyslipi_label = ["tg_cl", "tg_clf2", "tg_clf3",
                 "tc_cl", "tc_clf2", "tc_clf3",
                 "hdl_cl", "hdl_clf2", "hdl_clf3",
                 "ldl_cl", "ldl_clf2", "ldl_clf3",
                 "dys_cl", "dys_clf2", "dys_clf3"]

### Load and clean clinic information

In [ ]:
# process clinic data/ generate labels
df_clinic = pd.read_csv(clinic_data)
df_clinic["X"] = df_clinic.X.apply(lambda x: x[:2] + str(int(x[2:])))
df_clinic = df_clinic.set_index("X")
df_clinic[df_clinic >=999] = np.NaN

# impute na for med
med_list = ["DM_med", "hyper_med", "dyslipi_med"]
diagnosis_list = ["dm_cl", "hyper_cl", "dys_cl"]
cohorts = ["", "f2", "f3"]
for med, diag in zip(med_list, diagnosis_list):
    for cohort in cohorts:
        med_temp = med + cohort
        diag_temp = diag + cohort
        df_clinic.loc[df_clinic[diag_temp] == 0, med_temp] = 0
        df_clinic.loc[df_clinic[med_temp] == 5, med_temp] = 0
        df_clinic.loc[df_clinic[med_temp].isin([2, 3, 4]), med_temp] = 1

# remove unnecessary columns
df_clinic = df_clinic.loc[:, ~df_clinic.columns.isin(exclude_columns)]

Generate 10 years prediction label for followings:  
 - mets
 - hyper
 - dyslipi
 - dm  
 
The patient will be considered positive if any of the following criteria is met:  
 - the patient takes medicine at any point of time in the ten years
 - the patient is diagnosed as positive at cohort f2 or cohort f3

In [ ]:
# mets 10 years prediction
df_clinic["mets_10_years"] = np.NaN
df_clinic.loc[(df_clinic["metsf2"] == 1) | (df_clinic["metsf3"] == 1), "mets_10_years"] = 1
df_clinic.loc[(df_clinic["metsf2"] == 0) & (df_clinic["metsf3"] == 0), "mets_10_years"] = 0

# hyper 10 years prediction
df_clinic["hyper_cl_10_years"] = np.NaN
df_clinic.loc[(df_clinic["hyper_clf2"] == 1) | (df_clinic["hyper_clf3"] == 1), "hyper_cl_10_years"] = 1
df_clinic.loc[(df_clinic["hyper_clf2"] == 0) & (df_clinic["hyper_clf3"] == 0), "hyper_cl_10_years"] = 0

# dyslipi 10 years prediction
df_clinic["dys_cl_10_years"] = np.NaN
df_clinic.loc[(df_clinic["dys_clf2"] == 1) | (df_clinic["dys_clf3"] == 1), "dys_cl_10_years"] = 1
df_clinic.loc[(df_clinic["dys_clf2"] == 0) & (df_clinic["dys_clf3"] == 0), "dys_cl_10_years"] = 0

# dm 10 years prediction
df_clinic["dm_cl_10_years"] = np.NaN
df_clinic.loc[(df_clinic["dm_clf2"] == 1) | (df_clinic["dm_clf3"] == 1), "dm_cl_10_years"] = 1
df_clinic.loc[(df_clinic["dm_clf2"] == 0) & (df_clinic["dm_clf3"] == 0), "dm_cl_10_years"] = 0

label_10_years = ["mets_10_years", "dys_cl_10_years", "hyper_cl_10_years", "dm_cl_10_years"]
df_clinic

In [ ]:
df_clinic.loc[df_clinic.index == "NL1603", ["metsf2", "metsf2"]]

### Load and clean protein matrix

In [ ]:
# process baseline
df_prot_raw = pd.read_csv(prot_data_baseline)

# remove f2, f3  
df_prot_baseline_raw = df_prot_raw.loc[df_prot_raw.match.str.contains("baseline"), :]
df_prot_baseline_raw.match = df_prot_baseline_raw.match.apply(lambda x: x[:-8]) 

df_prot_baseline = df_prot_baseline_raw.copy()
df_prot_baseline.set_index("match", inplace=True)
# Define columns required
df_prot_baseline = df_prot_baseline.loc[:, ~df_prot_baseline.columns.isin(exclude_columns)]
df_prot_baseline[df_prot_baseline >=999] = np.NaN
df_prot_baseline = df_prot_baseline.loc[:, ~df_prot_baseline.columns.isin(clinic_measures + med_conditions + mets_label + dm_label + hyper_label + dyslipi_label)]
# remove samples with protein with missing rate over 30%
# df_prot = df_prot.loc[(df_prot.isna().sum(axis=1)/ (df_prot.shape[1]-1)) <= 0.5, :]
# extract protein list for future usage
# sincec the new data does not phase 4 data,
# we do not need to remove phase
protein_list_baseline = df_prot_baseline.columns.to_list()
# protein_list.remove("phase")

# df_prot_f2_raw =  df_prot_raw.loc[df_prot_raw.match.str.contains("F2"), :]
# df_prot_f2_raw.match = df_prot_f2_raw.match.apply(lambda x: x[:-8]) 
# df_prot_baseline_f2 = df_prot_f2_raw.copy()
# df_prot_baseline_f2.set_index("match", inplace=True)
# df_prot_baseline_f2 = df_prot_baseline_f2.loc[:, ~df_prot_baseline_f2.columns.isin(exclude_columns)]
# df_prot_baseline_f2[df_prot_baseline_f2 >=999] = np.NaN
# df_prot_baseline_f2 = df_prot_baseline_f2.loc[:, ~df_prot_baseline_f2.columns.isin(clinic_measures + med_conditions + mets_label + dm_label + hyper_label + dyslipi_label)]


# df_prot_f3_raw =  df_prot_raw.loc[df_prot_raw.match.str.contains("F3"), :]
# df_prot_f3_raw.match = df_prot_f3_raw.match.apply(lambda x: x[:-8]) 
# df_prot_baseline_f3 = df_prot_f3_raw.copy()
# df_prot_baseline_f3.set_index("match", inplace=True)
# df_prot_baseline_f3 = df_prot_baseline_f3.loc[:, ~df_prot_baseline_f3.columns.isin(exclude_columns)]
# df_prot_baseline_f3[df_prot_baseline_f3 >=999] = np.NaN
# df_prot_baseline_f3 = df_prot_baseline_f3.loc[:, ~df_prot_baseline_f3.columns.isin(clinic_measures + med_conditions + mets_label + dm_label + hyper_label + dyslipi_label)]

# check time consistency of protein distribution
# df_baseline_stable, _ = distribution_deviation(df_prot_baseline, df_prot_baseline_f2, threshold=0.01, method="ttest")
# protein_stable_list = df_baseline_stable.columns.to_list()
# df_baseline_stable, _ = distribution_deviation(df_baseline_stable, df_prot_baseline_f3[protein_stable_list], threshold=0.01, method="ttest")

#remove low variance features
# df_baseline_stable = df_baseline_stable.loc[:, df_baseline_stable.std(axis=0) > 0.5]
# df_prot_baseline = df_baseline_stable.copy()
df_prot_baseline

In [ ]:
# process perspective data
df_prot_perspective_raw = pd.read_csv(prot_data_perspective)

# remove f2, f3 
df_prot_perspective_raw = df_prot_perspective_raw.loc[df_prot_perspective_raw.ID.str.contains("baseline"), :]
df_prot_perspective_raw.ID = df_prot_perspective_raw.ID.apply(lambda x: x[:-8]) 
df_prot_perspective = df_prot_perspective_raw.copy()
df_prot_perspective.set_index("ID", inplace=True)
# Define columns required
df_prot_perspective = df_prot_perspective.loc[:, ~df_prot_perspective.columns.isin(exclude_columns)]
df_prot_perspective[df_prot_perspective >=999] = np.NaN
df_prot_perspective = df_prot_perspective.loc[:, ~df_prot_perspective.columns.isin(clinic_measures + med_conditions + mets_label + dm_label + hyper_label + dyslipi_label)]
# remove samples with protein with missing rate over 30%
# df_prot = df_prot.loc[(df_prot.isna().sum(axis=1)/ (df_prot.shape[1]-1)) <= 0.5, :]
# extract protein list for future usage
# sincec the new data does not phase 4 data,
# we do not need to remove phase
protein_list_perspective = df_prot_perspective.columns.to_list()
# protein_list.remove("phase")
df_prot_perspective

In [ ]:
threshold = 0.3
df_prot_baseline = df_prot_baseline.loc[:, (df_prot_baseline.isna().sum(axis=0) / df_prot_baseline.shape[0]) <= threshold]
protein_list = df_prot_baseline.columns.to_list()
extra_protein = list(set(df_prot_baseline.columns) - set(df_prot_perspective.columns))
df_prot_perspective[extra_protein] = 0
# print(extra_protein)
df_prot_perspective = df_prot_perspective[protein_list]
print(df_prot_baseline.shape, df_prot_perspective.shape)
len(set(df_prot_perspective.columns).intersection(set(df_prot_baseline.columns)))

### check the temporal stability of protien

In [ ]:
# # select proteins that are stable overtime
# df_baseline_stable, df_perspective_stable = distribution_deviation(df_prot_baseline[common_features], df_prot_perspective[common_features], threshold=0.05, method="ttest")
# df_baseline_stable.shape, df_perspective_stable.shape

### combine protein and clinic matrix
Generate the master table which contains all the information required for all downstream analysis.

In [ ]:
# construct the master table for baseline
df_master_baseline = df_prot_baseline.merge(df_clinic, how="inner", left_index=True, right_index=True)
df_master_baseline = df_master_baseline[protein_list + clinic_measures + med_conditions + dm_label + dyslipi_label + hyper_label + mets_label + label_10_years]

# construct the master table for perspective
df_master_perspective = df_prot_perspective.merge(df_clinic, how="inner", left_index=True, right_index=True)
df_master_perspective = df_master_perspective[protein_list + clinic_measures + med_conditions + dm_label + dyslipi_label + hyper_label + mets_label + label_10_years]
print(df_master_baseline.shape, df_master_perspective.shape)

### mets matrix

In [ ]:
# extract baseline matrix with mets label
df_baseline_mets_combined = df_master_baseline.dropna(subset=["mets_10_years"]).loc[df_master_baseline.mets == 0, protein_list + clinic_measures + ["mets_10_years"]]
df_baseline_mets_combined.rename(columns={"mets_10_years" : "label"}, inplace=True)
df_baseline_mets_clinic = df_master_baseline.dropna(subset=["mets_10_years"]).loc[df_master_baseline.mets == 0, clinic_measures + ["mets_10_years"]]
df_baseline_mets_clinic.rename(columns={"mets_10_years" : "label"}, inplace=True)
df_baseline_mets_protein = df_master_baseline.dropna(subset=["mets_10_years"]).loc[df_master_baseline.mets == 0, protein_list + ["mets_10_years"]]
df_baseline_mets_protein.rename(columns={"mets_10_years" : "label"}, inplace=True)

df_baseline_mets_combined.to_csv(TRANSFORM/ "mets/mets_combined_baseline_bem.csv")
df_baseline_mets_clinic.to_csv(TRANSFORM/ "mets/mets_clinic_baseline_bem.csv")
df_baseline_mets_protein.to_csv(TRANSFORM/ "mets/mets_protein_baseline_bem.csv")

print(df_baseline_mets_combined.shape, df_baseline_mets_clinic.shape, df_baseline_mets_protein.shape)
df_baseline_mets_combined.label.value_counts()

In [ ]:
# extract perspective matrix with mets label
df_perspective_mets_combined = df_master_perspective.dropna(subset=["mets_10_years"]).loc[df_master_perspective.mets == 0, protein_list + clinic_measures + ["mets_10_years"]]
df_perspective_mets_combined.rename(columns={"mets_10_years" : "label"}, inplace=True)
df_perspective_mets_clinic = df_master_perspective.dropna(subset=["mets_10_years"]).loc[df_master_perspective.mets == 0, clinic_measures + ["mets_10_years"]]
df_perspective_mets_clinic.rename(columns={"mets_10_years" : "label"}, inplace=True)
df_perspective_mets_protein = df_master_perspective.dropna(subset=["mets_10_years"]).loc[df_master_perspective.mets == 0, protein_list + ["mets_10_years"]]
df_perspective_mets_protein.rename(columns={"mets_10_years" : "label"}, inplace=True)

df_perspective_mets_combined.to_csv(TRANSFORM/ "mets/mets_combined_perspective.csv")
df_perspective_mets_clinic.to_csv(TRANSFORM/ "mets/mets_clinic_perspective.csv")
df_perspective_mets_protein.to_csv(TRANSFORM/ "mets/mets_protein_perspective.csv")

print(df_perspective_mets_combined.shape, df_perspective_mets_clinic.shape, df_perspective_mets_protein.shape)
df_perspective_mets_combined.label.value_counts()

#### mets results analysis

In [ ]:
# find the best clinic only model for mets
mets_clinic_result = pd.read_csv(root_dir / "results/2022/mets/clinic/true/no_sampling/metric_results.csv")
mets_clinic_result.sort_values(by="auc_perspective", ascending=False).head(2)

In [ ]:
# find the best protein only model for mets
mets_combined_result = pd.read_csv(root_dir / "results/2022/mets/protein/true/no_sampling/metric_results.csv")
mets_combined_result.sort_values(by="auc_perspective", ascending=False).head(2)

In [ ]:
# find the best combined model for mets
mets_combined_result = pd.read_csv(root_dir / "results/2022/mets/combined/true/no_sampling/metric_results.csv")
mets_combined_result.sort_values(by="auc_perspective", ascending=False).head(2)

In [40]:
# find the best clinic only model for mets
mets_clinic_result = pd.read_csv(root_dir / "results/2022/mets/clinic/true/no_sampling/metric_results.csv")
mets_clinic_result.sort_values(by="auc_perspective", ascending=False).head(2)
# find the best protein only model for mets
mets_protein_result = pd.read_csv(root_dir / "results/2022/mets/protein/true/no_sampling/metric_results.csv")
mets_protein_result.sort_values(by="auc_perspective", ascending=False).head(2)
# find the best combined model for mets
mets_combined_result = pd.read_csv(root_dir / "results/2022/mets/combined/true/no_sampling/metric_results.csv")
mets_combined_result.sort_values(by="auc_perspective", ascending=False).head(2)

# result = mets_combined_result[["auc_perspective"]] - mets_clinic_result[["auc_perspective"]]
result = mets_combined_result.merge(mets_clinic_result, on="seed")
result = result.merge(mets_protein_result, on="seed")
# result["combined_auc"] = mets_combined_result[["auc_perspective"]]
# result["clinic_auc"] = mets_clinic_result[["auc_perspective"]]
# result["seed"] = mets_combined_result["seed"].astype(str)
# result.sort_values(by="auc_perspective", ascending=False).head(20)
# sns.histplot(result.auc_perspective)
# BMI,wc,Glu,TG,HDL,SBP,P04004_VTN,P04278_SHBG,P43652_AFM,Q460N5_PARP14
result["auc_dif"] = result["auc_perspective_x"] - result["auc_perspective_y"]
result.rename(columns={"auc_perspective_x": "combiend_auc", "auc_baseline_x": "combiend_baseline", "auc_perspective_y": "clinic_auc", "auc_baseline_y": "clinic_baseline","auc_perspective":"protein_auc", "auc_baseline":"protein_baseline"}, inplace=True)
result = result[["seed", "auc_dif", "combiend_auc", "clinic_auc", "protein_auc","combiend_baseline", "clinic_baseline", "protein_baseline"]]
result.sort_values(by="auc_dif", ascending=False).head(50).reset_index(drop=True)

seed   auc_dif  combiend_auc  clinic_auc  protein_auc  combiend_baseline  \
0    279  0.047500      0.859643    0.812143     0.660060           0.862975   
1     89  0.043214      0.829762    0.786548     0.651250           0.711801   
2    995  0.043095      0.822143    0.779048     0.631726           0.687200   
3    520  0.042619      0.871548    0.828929     0.628810           0.790839   
4    349  0.041786      0.800357    0.758571     0.630000           0.755041   
5    490  0.040000      0.856548    0.816548     0.623155           0.798272   
6    486  0.039524      0.837619    0.798095     0.693929           0.765676   
7    603  0.036429      0.863571    0.827143     0.596190           0.789028   
8     30  0.036071      0.870833    0.834762     0.692381           0.795895   
9    202  0.035476      0.840357    0.804881     0.660655           0.788725   
10   148  0.034762      0.859405    0.824643     0.701905           0.821869   
11   295  0.034286      0.803214    0.768929     0.646310           0.817555   
12   675  0.033095      0.819881    0.786786     0.741310           0.799937   
13   268  0.030714      0.865476    0.834762     0.615000           0.789781   
14   513  0.030595      0.829405    0.798810     0.683571           0.814542   
15   197  0.030357      0.801548    0.771190     0.682024           0.753882   
16   556  0.030357      0.858929    0.828571     0.617738           0.817868   
17   887  0.029405      0.850357    0.820952     0.709286           0.783874   
18   850  0.028929      0.835714    0.806786     0.629524           0.806699   
19   811  0.028810      0.832024    0.803214     0.713214           0.825698   
20   346  0.027857      0.847738    0.819881     0.709643           0.836131   
21   707  0.027262      0.866190    0.838929     0.610714           0.826012   
22   636  0.027143      0.857857    0.830714     0.619643           0.831348   
23   614  0.026905      0.856071    0.829167     0.667381           0.797686   
24    41  0.026786      0.866310    0.839524     0.663929           0.809287   
25   580  0.026190      0.859167    0.832976     0.731071           0.769391   
26   711  0.025952      0.841190    0.815238     0.645833           0.785165   
27   234  0.025833      0.843452    0.817619     0.674048           0.737908   
28   409  0.025714      0.871786    0.846071     0.701548           0.795663   
29   994  0.023929      0.850476    0.826548     0.558214           0.832267   
30   221  0.023810      0.854762    0.830952     0.639881           0.818431   
31   709  0.023452      0.839881    0.816429     0.667738           0.786036   
32   930  0.023214      0.830238    0.807024     0.673690           0.756426   
33   646  0.023095      0.853929    0.830833     0.600952           0.763064   
34   625  0.022262      0.869286    0.847024     0.656429           0.839600   
35   535  0.021905      0.850357    0.828452     0.713869           0.794377   
36   144  0.021667      0.869762    0.848095     0.643690           0.780556   
37   835  0.021548      0.840476    0.818929     0.580238           0.808443   
38   595  0.021429      0.831905    0.810476     0.657679           0.774780   
39   781  0.021429      0.822143    0.800714     0.624881           0.811326   
40   989  0.020952      0.861190    0.840238     0.634940           0.826511   
41    87  0.020833      0.872738    0.851905     0.728929           0.871958   
42   881  0.020714      0.876071    0.855357     0.698452           0.812763   
43    27  0.020714      0.868095    0.847381     0.691071           0.795642   
44   652  0.020238      0.829643    0.809405     0.684881           0.816251   
45   613  0.020000      0.834048    0.814048     0.640060           0.753292   
46   672  0.019881      0.846667    0.826786     0.690714           0.819219   
47   716  0.019762      0.810238    0.790476     0.627143           0.801773   
48   836  0.018929      0.820476    0.801548     0.655774           0.805799   
49  

In [ ]:
# candidate 41/87/148/672/881

### met matrix with sub-labels  
#### This section will explore the potential ensemble model for mets prediction that combines 10 models that predict each of the critria of mets that list below:  
1. waist
2. glu
3. tg
4. hdl
5. sbpdbp

In [ ]:
def mets_sublabel_dataset_generator(dataset: pd.DataFrame, label: str, cohort:str)-> pd.DataFrame:
    """ this function will take the master datasets (i.e. baseline, perspective) and 
    output the corresponding sub label
    :param dataset: the master datasets
    :param label: the sublabel column name
    """
    baseline_label = label[:-2]
    print(label, cohort)
    df_baseline__combined = dataset.dropna(subset=[label]).loc[dataset[baseline_label] == 0, protein_list + clinic_measures + [label]]
    df_baseline__combined.rename(columns={label : "label"}, inplace=True)
    df_baseline__clinic = dataset.dropna(subset=[label]).loc[dataset[baseline_label] == 0, clinic_measures + [label]]
    df_baseline__clinic.rename(columns={label : "label"}, inplace=True)
    df_baseline__protein = dataset.dropna(subset=[label]).loc[dataset[baseline_label] == 0, protein_list + [label]]
    df_baseline__protein.rename(columns={label : "label"}, inplace=True)

    df_baseline__combined.to_csv(TRANSFORM/ f"mets/sub/mets_{label}_combined_{cohort}.csv")
    df_baseline__clinic.to_csv(TRANSFORM/ f"mets/sub/mets_{label}_clinic_{cohort}.csv")
    df_baseline__protein.to_csv(TRANSFORM/ f"mets/sub/mets_{label}_protein_{cohort}.csv")

    print(df_baseline__combined.label.value_counts())
    print("positive label proportion:")
    print((df_baseline__combined.label == 1).sum() / df_baseline__combined.shape[0])
    print()

In [ ]:
labels = ["waistf2", "fbgf2", "tgf2", "hdlf2", "sbpdbpf2", "waistf3", "fbgf3", "tgf3", "hdlf3", "sbpdbpf3"]
for label in labels:
    mets_sublabel_dataset_generator(df_master_baseline, label, "baseline")
    mets_sublabel_dataset_generator(df_master_perspective, label, "perspective")

In [ ]:
# selected_metric = "auc_baseline" 
selected_metric = "auc_cv" 

#### waistf2

In [ ]:
# clinic
mets_waistf2_clinic_result = pd.read_csv(root_dir / "results/2022/mets/sub/waistf2/clinic/true/no_sampling/metric_results.csv")
mets_waistf2_clinic_result.sort_values(by=selected_metric, ascending=False).head(1)

In [ ]:
# protein
mets_waistf2_protein_result = pd.read_csv(root_dir / "results/2022/mets/sub/waistf2/protein/true/no_sampling/metric_results.csv")
mets_waistf2_protein_result.sort_values(by=selected_metric, ascending=False).head(1)

In [ ]:
# combined
mets_waistf2_combined_result = pd.read_csv(root_dir / "results/2022/mets/sub/waistf2/combined/true/no_sampling/metric_results.csv")
mets_waistf2_combined_result.sort_values(by=selected_metric, ascending=False).head(1)

#### waistf3

In [ ]:
# clinic
mets_waistf3_clinic_result = pd.read_csv(root_dir / "results/2022/mets/sub/waistf3/clinic/true/no_sampling/metric_results.csv")
mets_waistf3_clinic_result.sort_values(by=selected_metric, ascending=False).head(1)

In [ ]:
# protein
mets_waistf3_protein_result = pd.read_csv(root_dir / "results/2022/mets/sub/waistf3/protein/true/no_sampling/metric_results.csv")
mets_waistf3_protein_result.sort_values(by=selected_metric, ascending=False).head(1)

In [ ]:
# combined
mets_waistf3_combined_result = pd.read_csv(root_dir / "results/2022/mets/sub/waistf3/combined/true/no_sampling/metric_results.csv")
mets_waistf3_combined_result.sort_values(by=selected_metric, ascending=False).head(1)

#### fbgf2

In [ ]:
# clinic
mets_fbgf2_clinic_result = pd.read_csv(root_dir / "results/2022/mets/sub/fbgf2/clinic/true/no_sampling/metric_results.csv")
mets_fbgf2_clinic_result.sort_values(by=selected_metric, ascending=False).head(1)

In [ ]:
# protein
mets_fbgf2_protein_result = pd.read_csv(root_dir / "results/2022/mets/sub/fbgf2/protein/true/no_sampling/metric_results.csv")
mets_fbgf2_protein_result.sort_values(by=selected_metric, ascending=False).head(1)

In [ ]:
# combined
mets_fbgf2_combined_result = pd.read_csv(root_dir / "results/2022/mets/sub/fbgf2/combined/true/no_sampling/metric_results.csv")
mets_fbgf2_combined_result.sort_values(by=selected_metric, ascending=False).head(1)

#### fbgf3

In [ ]:
# clinic
mets_fbgf3_clinic_result = pd.read_csv(root_dir / "results/2022/mets/sub/fbgf3/clinic/true/no_sampling/metric_results.csv")
mets_fbgf3_clinic_result.sort_values(by=selected_metric, ascending=False).head(1)

In [ ]:
# protein
mets_fbgf3_protein_result = pd.read_csv(root_dir / "results/2022/mets/sub/fbgf3/protein/true/no_sampling/metric_results.csv")
mets_fbgf3_protein_result.sort_values(by=selected_metric, ascending=False).head(1)

In [ ]:
# combined
mets_fbgf3_combined_result = pd.read_csv(root_dir / "results/2022/mets/sub/fbgf3/combined/true/no_sampling/metric_results.csv")
mets_fbgf3_combined_result.sort_values(by=selected_metric, ascending=False).head(1)

#### tgf2

In [ ]:
# clinic
mets_tgf2_clinic_result = pd.read_csv(root_dir / "results/2022/mets/sub/tgf2/clinic/true/no_sampling/metric_results.csv")
mets_tgf2_clinic_result.sort_values(by=selected_metric, ascending=False).head(1)

In [ ]:
# protein
mets_tgf2_protein_result = pd.read_csv(root_dir / "results/2022/mets/sub/tgf2/protein/true/no_sampling/metric_results.csv")
mets_tgf2_protein_result.sort_values(by=selected_metric, ascending=False).head(1)

In [ ]:
# combined
mets_tgf2_combined_result = pd.read_csv(root_dir / "results/2022/mets/sub/tgf2/combined/true/no_sampling/metric_results.csv")
mets_tgf2_combined_result.sort_values(by=selected_metric, ascending=False).head(1)

#### tgf3

In [ ]:
# clinic
mets_tgf3_clinic_result = pd.read_csv(root_dir / "results/2022/mets/sub/tgf3/clinic/true/no_sampling/metric_results.csv")
mets_tgf3_clinic_result.sort_values(by=selected_metric, ascending=False).head(1)

In [ ]:
# protein
mets_tgf3_protein_result = pd.read_csv(root_dir / "results/2022/mets/sub/tgf3/protein/true/no_sampling/metric_results.csv")
mets_tgf3_protein_result.sort_values(by=selected_metric, ascending=False).head(1)

In [ ]:
# combined
mets_tgf3_combined_result = pd.read_csv(root_dir / "results/2022/mets/sub/tgf3/combined/true/no_sampling/metric_results.csv")
mets_tgf3_combined_result.sort_values(by=selected_metric, ascending=False).head(1)

#### hdlf2

In [ ]:
# clinic
mets_hdlf2_clinic_result = pd.read_csv(root_dir / "results/2022/mets/sub/hdlf2/clinic/true/no_sampling/metric_results.csv")
mets_hdlf2_clinic_result.sort_values(by=selected_metric, ascending=False).head(1)

In [ ]:
# protein
mets_hdlf2_protein_result = pd.read_csv(root_dir / "results/2022/mets/sub/hdlf2/protein/true/no_sampling/metric_results.csv")
mets_hdlf2_protein_result.sort_values(by=selected_metric, ascending=False).head(1)

In [ ]:
# combined
mets_hdlf2_combined_result = pd.read_csv(root_dir / "results/2022/mets/sub/hdlf2/combined/true/no_sampling/metric_results.csv")
mets_hdlf2_combined_result.sort_values(by=selected_metric, ascending=False).head(1)

#### hdlf3

In [ ]:
# clinic
mets_hdlf3_clinic_result = pd.read_csv(root_dir / "results/2022/mets/sub/hdlf3/clinic/true/no_sampling/metric_results.csv")
mets_hdlf3_clinic_result.sort_values(by=selected_metric, ascending=False).head(1)

In [ ]:
# protein
mets_hdlf3_protein_result = pd.read_csv(root_dir / "results/2022/mets/sub/hdlf3/protein/true/no_sampling/metric_results.csv")
mets_hdlf3_protein_result.sort_values(by=selected_metric, ascending=False).head(1)

In [ ]:
# combined
mets_hdlf3_combined_result = pd.read_csv(root_dir / "results/2022/mets/sub/hdlf3/combined/true/no_sampling/metric_results.csv")
mets_hdlf3_combined_result.sort_values(by=selected_metric, ascending=False).head(1)

#### sbpdbpf2

In [ ]:
# clinic
mets_sbpdbpf2_clinic_result = pd.read_csv(root_dir / "results/2022/mets/sub/sbpdbpf2/clinic/true/no_sampling/metric_results.csv")
mets_sbpdbpf2_clinic_result.sort_values(by=selected_metric, ascending=False).head(1)

In [ ]:
# protein
mets_sbpdbpf2_protein_result = pd.read_csv(root_dir / "results/2022/mets/sub/sbpdbpf2/protein/true/no_sampling/metric_results.csv")
mets_sbpdbpf2_protein_result.sort_values(by=selected_metric, ascending=False).head(1)

In [ ]:
# combined
mets_sbpdbpf2_combined_result = pd.read_csv(root_dir / "results/2022/mets/sub/sbpdbpf2/combined/true/no_sampling/metric_results.csv")
mets_sbpdbpf2_combined_result.sort_values(by=selected_metric, ascending=False).head(1)

#### sbpdbpf3

In [ ]:
# clinic
mets_sbpdbpf3_clinic_result = pd.read_csv(root_dir / "results/2022/mets/sub/sbpdbpf3/clinic/true/no_sampling/metric_results.csv")
mets_sbpdbpf3_clinic_result.sort_values(by=selected_metric, ascending=False).head(1)

In [ ]:
# protein
mets_sbpdbpf3_protein_result = pd.read_csv(root_dir / "results/2022/mets/sub/sbpdbpf3/protein/true/no_sampling/metric_results.csv")
mets_sbpdbpf3_protein_result.sort_values(by=selected_metric, ascending=False).head(1)

In [ ]:
# combined
mets_sbpdbpf3_combined_result = pd.read_csv(root_dir / "results/2022/mets/sub/sbpdbpf3/combined/true/no_sampling/metric_results.csv")
mets_sbpdbpf3_combined_result.sort_values(by=selected_metric, ascending=False).head(1)

### dyslipi matrix

In [ ]:
# extract matrix with dyslipi label
df_baseline_dyslipi_combined = df_master_baseline.dropna(subset=["dys_cl_10_years"]).loc[df_master_baseline.dys_cl == 0, protein_list + clinic_measures + ["dys_cl_10_years"]]
df_baseline_dyslipi_combined.rename(columns={"dys_cl_10_years" : "label"}, inplace=True)
df_baseline_dyslipi_clinic = df_master_baseline.dropna(subset=["dys_cl_10_years"]).loc[df_master_baseline.dys_cl == 0, clinic_measures + ["dys_cl_10_years"]]
df_baseline_dyslipi_clinic.rename(columns={"dys_cl_10_years" : "label"}, inplace=True)
df_baseline_dyslipi_protein = df_master_baseline.dropna(subset=["dys_cl_10_years"]).loc[df_master_baseline.dys_cl == 0, protein_list + ["dys_cl_10_years"]]
df_baseline_dyslipi_protein.rename(columns={"dys_cl_10_years" : "label"}, inplace=True)

df_baseline_dyslipi_combined.to_csv(TRANSFORM/ "dyslipi/dyslipi_combined_baseline.csv")
df_baseline_dyslipi_clinic.to_csv(TRANSFORM/ "dyslipi/dyslipi_clinic_baseline.csv")
df_baseline_dyslipi_protein.to_csv(TRANSFORM/ "dyslipi/dyslipi_protein_baseline.csv")

df_baseline_dyslipi_combined.label.value_counts()

In [ ]:
# extract matrix with dyslipi label
df_perspective_dyslipi_combined = df_master_perspective.dropna(subset=["dys_cl_10_years"]).loc[df_master_perspective.dys_cl == 0, protein_list + clinic_measures + ["dys_cl_10_years"]]
df_perspective_dyslipi_combined.rename(columns={"dys_cl_10_years" : "label"}, inplace=True)
df_perspective_dyslipi_clinic = df_master_perspective.dropna(subset=["dys_cl_10_years"]).loc[df_master_perspective.dys_cl == 0, clinic_measures + ["dys_cl_10_years"]]
df_perspective_dyslipi_clinic.rename(columns={"dys_cl_10_years" : "label"}, inplace=True)
df_perspective_dyslipi_protein = df_master_perspective.dropna(subset=["dys_cl_10_years"]).loc[df_master_perspective.dys_cl == 0, protein_list + ["dys_cl_10_years"]]
df_perspective_dyslipi_protein.rename(columns={"dys_cl_10_years" : "label"}, inplace=True)

df_perspective_dyslipi_combined.to_csv(TRANSFORM/ "dyslipi/dyslipi_combined_perspective.csv")
df_perspective_dyslipi_clinic.to_csv(TRANSFORM/ "dyslipi/dyslipi_clinic_perspective.csv")
df_perspective_dyslipi_protein.to_csv(TRANSFORM/ "dyslipi/dyslipi_protein_perspective.csv")

df_perspective_dyslipi_combined.label.value_counts()

#### dyslipi result analysis

In [ ]:
# find the best clinic only model for dyslipi
dyslipi_clinic_result = pd.read_csv(root_dir / "results/2022/dyslipi/clinic/true/no_sampling/metric_results.csv")
dyslipi_clinic_result.sort_values(by="auc_cv", ascending=False).head(1)

In [ ]:
# find the best protein only model for dyslipi
dyslipi_protein_result = pd.read_csv(root_dir / "results/2022/dyslipi/protein/true/no_sampling/metric_results.csv")
dyslipi_protein_result.sort_values(by="auc_cv", ascending=False).head(1)

In [ ]:
# find the best combined model for dyslipi
dyslipi_combined_result = pd.read_csv(root_dir / "results/2022/dyslipi/combined/true/no_sampling/metric_results.csv")
dyslipi_combined_result.sort_values(by="auc_cv", ascending=False).head(1)

### hyper matrix

In [ ]:
df_baseline_hyper_combined = df_master_baseline.dropna(subset=["hyper_cl_10_years"]).loc[df_master_baseline.hyper_cl == 0, protein_list + clinic_measures + ["hyper_cl_10_years"]]
df_baseline_hyper_combined.rename(columns={"hyper_cl_10_years" : "label"}, inplace=True)
df_baseline_hyper_clinic = df_master_baseline.dropna(subset=["hyper_cl_10_years"]).loc[df_master_baseline.hyper_cl == 0, clinic_measures + ["hyper_cl_10_years"]]
df_baseline_hyper_clinic.rename(columns={"hyper_cl_10_years" : "label"}, inplace=True)
df_baseline_hyper_protein = df_master_baseline.dropna(subset=["hyper_cl_10_years"]).loc[df_master_baseline.hyper_cl == 0, protein_list + ["hyper_cl_10_years"]]
df_baseline_hyper_protein.rename(columns={"hyper_cl_10_years" : "label"}, inplace=True)

df_baseline_hyper_combined.to_csv(TRANSFORM/ "hyper/hyper_combined_baseline.csv")
df_baseline_hyper_clinic.to_csv(TRANSFORM/ "hyper/hyper_clinic_baseline.csv")
df_baseline_hyper_protein.to_csv(TRANSFORM/ "hyper/hyper_protein_baseline.csv")

df_baseline_hyper_combined.label.value_counts()

In [ ]:
df_perspective_hyper_combined = df_master_perspective.dropna(subset=["hyper_cl_10_years"]).loc[df_master_perspective.hyper_cl == 0, protein_list + clinic_measures + ["hyper_cl_10_years"]]
df_perspective_hyper_combined.rename(columns={"hyper_cl_10_years" : "label"}, inplace=True)
df_perspective_hyper_clinic = df_master_perspective.dropna(subset=["hyper_cl_10_years"]).loc[df_master_perspective.hyper_cl == 0, clinic_measures + ["hyper_cl_10_years"]]
df_perspective_hyper_clinic.rename(columns={"hyper_cl_10_years" : "label"}, inplace=True)
df_perspective_hyper_protein = df_master_perspective.dropna(subset=["hyper_cl_10_years"]).loc[df_master_perspective.hyper_cl == 0, protein_list + ["hyper_cl_10_years"]]
df_perspective_hyper_protein.rename(columns={"hyper_cl_10_years" : "label"}, inplace=True)

df_perspective_hyper_combined.to_csv(TRANSFORM/ "hyper/hyper_combined_perspective.csv")
df_perspective_hyper_clinic.to_csv(TRANSFORM/ "hyper/hyper_clinic_perspective.csv")
df_perspective_hyper_protein.to_csv(TRANSFORM/ "hyper/hyper_protein_perspective.csv")

df_perspective_hyper_combined.label.value_counts()

#### hyper results analysis

In [ ]:
# find the best clinic only model for dm
hyper_clinic_result = pd.read_csv(root_dir / "results/2022/hyper/clinic/true/no_sampling/metric_results.csv")
hyper_clinic_result.sort_values(by="auc_cv", ascending=False).head(1)

In [ ]:
# find the best protein only model for dm
hyper_protein_result = pd.read_csv(root_dir / "results/2022/hyper/protein/true/no_sampling/metric_results.csv")
hyper_protein_result.sort_values(by="auc_cv", ascending=False).head(1)

In [ ]:
hyper_combined_result = pd.read_csv(root_dir / "results/2022/hyper/combined/true/no_sampling/metric_results.csv")
hyper_combined_result.sort_values(by="auc_cv", ascending=False).head(1)

### dm matrix

In [ ]:
df_baseline_dm_combined = df_master_baseline.dropna(subset=["dm_cl_10_years"]).loc[df_master_baseline.dm_cl == 0, protein_list + clinic_measures + ["dm_cl_10_years"]]
df_baseline_dm_combined.rename(columns={"dm_cl_10_years" : "label"}, inplace=True)
df_baseline_dm_clinic = df_master_baseline.dropna(subset=["dm_cl_10_years"]).loc[df_master_baseline.dm_cl == 0, clinic_measures + ["dm_cl_10_years"]]
df_baseline_dm_clinic.rename(columns={"dm_cl_10_years" : "label"}, inplace=True)
df_baseline_dm_protein = df_master_baseline.dropna(subset=["dm_cl_10_years"]).loc[df_master_baseline.dm_cl == 0, protein_list + ["dm_cl_10_years"]]
df_baseline_dm_protein.rename(columns={"dm_cl_10_years" : "label"}, inplace=True)

df_baseline_dm_combined.to_csv(TRANSFORM/ "dm/dm_combined_baseline.csv")
df_baseline_dm_clinic.to_csv(TRANSFORM/ "dm/dm_clinic_baseline.csv")
df_baseline_dm_protein.to_csv(TRANSFORM/ "dm/dm_protein_baseline.csv")
df_baseline_dm_combined.label.value_counts()

In [ ]:
df_perspective_dm_combined = df_master_perspective.dropna(subset=["dm_cl_10_years"]).loc[df_master_perspective.dm_cl == 0, protein_list + clinic_measures + ["dm_cl_10_years"]]
df_perspective_dm_combined.rename(columns={"dm_cl_10_years" : "label"}, inplace=True)
df_perspective_dm_clinic = df_master_perspective.dropna(subset=["dm_cl_10_years"]).loc[df_master_perspective.dm_cl == 0, clinic_measures + ["dm_cl_10_years"]]
df_perspective_dm_clinic.rename(columns={"dm_cl_10_years" : "label"}, inplace=True)
df_perspective_dm_protein = df_master_perspective.dropna(subset=["dm_cl_10_years"]).loc[df_master_perspective.dm_cl == 0, protein_list + ["dm_cl_10_years"]]
df_perspective_dm_protein.rename(columns={"dm_cl_10_years" : "label"}, inplace=True)

df_perspective_dm_combined.to_csv(TRANSFORM/ "dm/dm_combined_perspective.csv")
df_perspective_dm_clinic.to_csv(TRANSFORM/ "dm/dm_clinic_perspective.csv")
df_perspective_dm_protein.to_csv(TRANSFORM/ "dm/dm_protein_perspective.csv")
df_perspective_dm_combined.label.value_counts()

In [ ]:
# find the best clinic only model for dm
dm_clinic_result = pd.read_csv(root_dir / "results/2022/dm/clinic/true/no_sampling/metric_results.csv")
dm_clinic_result.sort_values(by="auc_cv", ascending=False).head(1)

In [ ]:
# find the best protein only model for dm
dm_protein_result = pd.read_csv(root_dir / "results/2022/dm/protein/true/no_sampling/metric_results.csv")
dm_protein_result.sort_values(by="auc_cv", ascending=False).head(1)

In [ ]:
# find the best protein only model for dm
dm_protein_result = pd.read_csv(root_dir / "results/2022/dm/combined/true/no_sampling/metric_results.csv")
dm_protein_result.sort_values(by="auc_cv", ascending=False).head(1)

In [ ]:
# check colinearity
# df_baseline_met_no_med = df_baseline_met.loc[:, ~df_baseline_met.columns.isin(["dyslipi_med","DM_med","hyper_med","label"])]
# vifs = pd.Series(np.linalg.inv(df_baseline_met_no_med.fillna(0).corr().to_numpy()).diagonal(), index=df_baseline_met_no_med.columns, name='VIF').dropna()
# df_baseline_met = df_baseline_met.loc[:, ~df_baseline_met.columns.isin(vifs[vifs > 10].index.values)]
# df_baseline_met

### Sub_labels generation
- 5 years labels: "waistf2", "dm_clf2", "tg_clf2", "hdl_clf2", "hyper_clf2", "tc_clf2", "ldl_clf2", "dys_clf2"
- 10 year labels: "waistf3", "dm_clf3", "tg_clf3", "hdl_clf3", "hyper_clf3", "tc_clf3", "ldl_clf3", "dys_clf3"

In [ ]:
# process clinic data/ generate labels
df_clinic = pd.read_csv(clinic_data)
df_clinic["X"] = df_clinic.X.apply(lambda x: x[:2] + str(int(x[2:])))
df_clinic = df_clinic.set_index("X")
df_clinic[df_clinic >=999] = np.NaN
df_label_sublabel = df_clinic[["waistf2", "dm_clf2", "tg_clf2", "hdl_clf2", "hyper_clf2", "waistf3", "dm_clf3", "tg_clf3", "hdl_clf3", "hyper_clf3", "tc_clf2",  "tc_clf3", "ldl_clf2", "ldl_clf3", "dys_clf2", "dys_clf3"]]
df_label_sublabel

### Model Development Dataset Generation
The model development uses phase 1 data.
- 90% of phase 1 data is used for training
- 10% of phase 1 data is used for testing
- change of random seed will generate different results

In [ ]:
# train/test split (9/1)
df_master_, df_baseline_met_test = train_test_split(df_baseline_met, test_size = 0.1, random_state=seed)
df_baseline_train_label = df_label.loc[df_baseline_met_train.index, :]
df_baseline_test_label = df_label.loc[df_baseline_met_test.index, :]
df_perspective_label = df_label.loc[df_perspective.index, :]

df_baseline_train_label.shape, df_baseline_test_label.shape, df_perspective_label.shape

In [ ]:
df_perspective

### Generate dataset for baseline testing and perspective testing of met label

In [ ]:
# save internal test dataset for test big ensemble model
df_baseline_train_met = df_baseline_met.loc[df_baseline_met_train.index, :]
df_baseline_test_met = df_baseline_met.loc[df_baseline_met_test.index, :]

df_baseline_train_met.to_csv(root_dir/ "data/transformed_data/baseline_train_met.csv")
df_baseline_test_met.to_csv(root_dir/ "data/transformed_data/baseline_test_met.csv")

In [ ]:
df_baseline_train_label

In [ ]:
labels = ["waistf2", "fbgf2", "tgf2", "hdlf2", "sbpdbpf2", "waistf3", "fbgf3", "tgf3", "hdlf3", "sbpdbpf3"]

# generate training datasets for sublabels
for label in labels:
    df = sub_label_generator(df_train, df_train_label, label)
    df.to_csv(f"data/transformed_data/sub_label/baseline_combined_all_{label}.csv")

In [ ]:
# generate internal validation datasets
for label in labels:
    df = sub_label_generator(df_test, df_test_label, label)
    df.to_csv(f"data/transformed_data/sub_label/baseline_validation_combined_all_{label}.csv")

### Perpective (Valid) Dataset Generation
The following blocks will generate the dataset that used for validation (Phase 4).

In [ ]:
df_model_val = df_prot.loc[df_prot["phase"] == 4, df_prot.columns != "phase"]
df_model_val.to_csv("data/transformed_data/perspective_validation_combined_all.csv")
df_model_val.label.value_counts()

In [ ]:
df_model_val_label = df_label.loc[df_model_val.index]

for label in labels:
    df = sublabel_generator(df_model_val, df_model_val_label, label)
    df.to_csv(f"data/transformed_data/sub_label/perspective_validation_combined_all_{label}.csv")

### Training data with clinic information only  
columns included: "age", "BMI", "DBP", "Glu", "HDL", "hdl", "LDL", "SBP", "sex", "sbpdbp", "TG", "wc", "waist", "tg", "hdl", "fbg"

In [ ]:
clinic_cols = ["age", "BMI", "DBP", "Glu", "HDL", "LDL", "SBP", "sex", "TG", "wc", "DM_med", "hyper_med","label"]
df_train_clinic = df_train[clinic_cols]
df_test_clinic = df_test[clinic_cols]
df_model_val_clinic = df_model_val[clinic_cols]
df_train_clinic.to_csv("data/transformed_data/baseline_clinic_only.csv")
df_test_clinic.to_csv("data/transformed_data/baseline_validation_clinic_only.csv")
df_model_val_clinic.to_csv("data/transformed_data/perspective_validation_clinic_only.csv")

In [ ]:
df_train_clinic

In [ ]:
for label in labels:
    df = sublabel_generator(df_train_clinic, df_train_label, label)
    df.to_csv(f"data/transformed_data/sub_label/baseline_clinic_only_{label}.csv")

In [ ]:
for label in labels:
    df = sublabel_generator(df_test_clinic, df_test_label, label)
    df.to_csv(f"data/transformed_data/sub_label/baseline_validation_clinic_only_{label}.csv")

In [ ]:
for label in s:
    df = sublabel_generator(df_model_val_clinic, df_model_val_label, label)
    df.to_csv(f"data/transformed_data/sub_label/perspective_validation_clinic_only_{label}.csv")